In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

/home/djjin/anaconda3/envs/conda_visual_HPE/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
torch:  1.13 ; cuda:  cu117
detectron2: 0.6


In [2]:
import os
import sys
import logging
import argparse

pth = '/'.join(sys.path[0].split('/')[:-1])
sys.path.insert(0, pth)

from pprint import pprint
import numpy as np
np.random.seed(1)

home_dir = os.path.abspath(os.getcwd()+"/../")
sys.path.append(home_dir)
print(home_dir)

import warnings
warnings.filterwarnings(action='ignore')
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

from hdecoder.BaseModel import BaseModel
from hdecoder import build_model
from utils.distributed import init_distributed
from utils.arguments import load_opt_from_config_files, load_config_dict_to_opt

/home/djjin/Mygit/X-Decoder


DEBUG:git.cmd:Popen(['git', 'version'], cwd=/home/djjin/Mygit/X-Decoder/notebooks, universal_newlines=False, shell=None, istream=None)
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/home/djjin/Mygit/X-Decoder/notebooks, universal_newlines=False, shell=None, istream=None)
DEBUG:wandb.docker.auth:Trying paths: ['/home/djjin/.docker/config.json', '/home/djjin/.dockercfg']
DEBUG:wandb.docker.auth:No config file found
Invalid MIT-MAGIC-COOKIE-1 key

In [3]:
parser = argparse.ArgumentParser(description='Pretrain or fine-tune models for NLP tasks.')
parser.add_argument('--command', default="evaluate", help='Command: train/evaluate/train-and-evaluate')
parser.add_argument('--conf_files', nargs='+', help='Path(s) to the config file(s).')
parser.add_argument('--user_dir', help='Path to the user defined module for tasks (models, criteria), optimizers, and lr schedulers.')
parser.add_argument('--config_overrides', nargs='*', help='Override parameters on config with a json style string, e.g. {"<PARAM_NAME_1>": <PARAM_VALUE_1>, "<PARAM_GROUP_2>.<PARAM_SUBGROUP_2>.<PARAM_2>": <PARAM_VALUE_2>}. A key with "." updates the object in the corresponding nested dict. Remember to escape " in command line.')
parser.add_argument('--overrides', help='arguments that used to override the config file in cmdline', nargs=argparse.REMAINDER)

cmdline_args = parser.parse_args('')
# cmdline_args.conf_files = [os.path.join(home_dir, "configs/xdecoder/svlp_focalt_lang.yaml")]
cmdline_args.conf_files = [os.path.join(home_dir, "configs/xdecoder/vcoco.yaml")]
cmdline_args.overrides = ['WEIGHT', 'true', 'RESUME_FROM', '../checkpoints/xdecoder_focalt_best_openseg.pt'] 

opt = load_opt_from_config_files(cmdline_args.conf_files)

keys = [cmdline_args.overrides[idx*2] for idx in range(len(cmdline_args.overrides)//2)]
vals = [cmdline_args.overrides[idx*2+1] for idx in range(len(cmdline_args.overrides)//2)]
print(keys, vals)
vals = [val.replace('false', '').replace('False','') if len(val.replace(' ', '')) == 5 else val for val in vals]
types = []
for key in keys:
    key = key.split('.')
    ele = opt.copy()
    while len(key) > 0:
        ele = ele[key.pop(0)]
    types.append(type(ele))

config_dict = {x:z(y) for x,y,z in zip(keys, vals, types)}

load_config_dict_to_opt(opt, config_dict)
for key, val in cmdline_args.__dict__.items():
    if val is not None:
        opt[key] = val
opt = init_distributed(opt)

['WEIGHT', 'RESUME_FROM'] ['true', '../checkpoints/xdecoder_focalt_best_openseg.pt']


In [4]:
pretrained_pth = os.path.join(opt['RESUME_FROM'])
print(pretrained_pth)

../checkpoints/xdecoder_focalt_best_openseg.pt


In [5]:
model = BaseModel(opt, build_model(opt)).from_pretrained(pretrained_pth).eval().cuda()

INFO:utils.model:Loaded backbone.layers.0.blocks.0.dw1.bias, Model Shape: torch.Size([96]) <-> Ckpt Shape: torch.Size([96])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.dw1.weight, Model Shape: torch.Size([96, 1, 3, 3]) <-> Ckpt Shape: torch.Size([96, 1, 3, 3])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.dw2.bias, Model Shape: torch.Size([96]) <-> Ckpt Shape: torch.Size([96])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.dw2.weight, Model Shape: torch.Size([96, 1, 3, 3]) <-> Ckpt Shape: torch.Size([96, 1, 3, 3])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.gamma_1, Model Shape: torch.Size([96]) <-> Ckpt Shape: torch.Size([96])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.gamma_2, Model Shape: torch.Size([96]) <-> Ckpt Shape: torch.Size([96])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.mlp.fc1.bias, Model Shape: torch.Size([384]) <-> Ckpt Shape: torch.Size([384])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.mlp.fc1.weight, Model Shape: torch.Si

In [6]:
from datasets import build_evaluator, build_eval_dataloader
dataloaders = build_eval_dataloader(opt)
dataset_names = opt["DATASETS"]["TEST"]
opt['GRADIENT_ACCUMULATE_STEP'] = int(opt.get('GRADIENT_ACCUMULATE_STEP', 1))
opt['LR_SCHEDULER_PARAMS'] = opt.get('LR_SCHEDULER_PARAMS', {})

INFO:detectron2.data.common:Serializing 4946 elements to byte tensors and concatenating them all ...
INFO:detectron2.data.common:Serialized dataset takes 3.27 MiB


In [7]:
def get_dataloaders(
    dataset_label: str,
    is_evaluation: bool
):
    if is_evaluation:

        dataloaders = build_eval_dataloader(opt)
        dataloader = dataloaders[0]

        # temp solution for lr scheduler
        steps_total = len(dataloader)
        steps_acc = opt['GRADIENT_ACCUMULATE_STEP']
        steps_update = steps_total // steps_acc
        opt["LR_SCHEDULER_PARAMS"]["steps_update_per_epoch"] = steps_update
    return dataloader

In [8]:
# init metadata
from datasets.evaluation import VCOCOEvaluator
from trainer.utils.misc import move_batch_to_device, cast_batch_to_half
from datasets.utils.misc import all_gather
from copy import deepcopy
import itertools
from detectron2.data import MetadataCatalog
from collections import defaultdict
from datasets.utils.misc import all_gather, MetricLogger

scores = {}
summary = {}
evaluator = VCOCOEvaluator(opt["DATASETS"]["TEST"][0], 
                           correct_mat_dir=MetadataCatalog.get(opt["DATASETS"]["TEST"][0]).correct_mat_dir,
                           output_dir=opt["SAVE_DIR"])


eval_type = model.model.metadata.evaluator_type
evaluator.reset()

metric_logger = MetricLogger(delimiter="  ")
header = 'Test:'

with torch.no_grad():
    for idx, batch in enumerate(metric_logger.log_every(dataloaders[0], 10, header)):
        outputs = model(batch, mode=eval_type)
        evaluator.process(batch, outputs)
    metric_logger.synchronize_between_processes()

map = evaluator.evaluate()

Test:  [   0/2473]  eta: 1:07:13    time: 1.6309  data: 0.0333  max mem: 1016
Test:  [  10/2473]  eta: 0:26:09    time: 0.6374  data: 0.0323  max mem: 1185
Test:  [  20/2473]  eta: 0:24:29    time: 0.5473  data: 0.0318  max mem: 1185
Test:  [  30/2473]  eta: 0:23:38    time: 0.5496  data: 0.0328  max mem: 1231
Test:  [  40/2473]  eta: 0:22:56    time: 0.5310  data: 0.0336  max mem: 1231
Test:  [  50/2473]  eta: 0:22:31    time: 0.5225  data: 0.0324  max mem: 1275
Test:  [  60/2473]  eta: 0:22:37    time: 0.5562  data: 0.0324  max mem: 1275
Test:  [  70/2473]  eta: 0:22:22    time: 0.5606  data: 0.0333  max mem: 1275
Test:  [  80/2473]  eta: 0:21:55    time: 0.5107  data: 0.0326  max mem: 1275
Test:  [  90/2473]  eta: 0:21:50    time: 0.5195  data: 0.0324  max mem: 1275
Test:  [ 100/2473]  eta: 0:21:40    time: 0.5397  data: 0.0322  max mem: 1275
Test:  [ 110/2473]  eta: 0:21:28    time: 0.5245  data: 0.0327  max mem: 1275
Test:  [ 120/2473]  eta: 0:21:32    time: 0.5565  data: 0.0336  

In [9]:
map

{'AP_hold_obj': 0,
 'AP_stand': 0.00018254837531945964,
 'AP_sit_instr': 0,
 'AP_ride_instr': 0,
 'AP_walk': 0,
 'AP_look_obj': 0.0,
 'AP_hit_instr': 0,
 'AP_hit_obj': 0,
 'AP_eat_obj': 0,
 'AP_eat_instr': 0.0,
 'AP_jump_instr': 0,
 'AP_lay_instr': 0,
 'AP_talk_on_phone_instr': 0,
 'AP_carry_obj': 0.0,
 'AP_throw_obj': 0,
 'AP_catch_obj': 0,
 'AP_cut_instr': 0,
 'AP_cut_obj': 0,
 'AP_run': 0.0,
 'AP_work_on_computer_instr': 0,
 'AP_ski_instr': 0.0,
 'AP_surf_instr': 0,
 'AP_skateboard_instr': 0,
 'AP_smile': 0,
 'AP_drink_instr': 0,
 'AP_kick_obj': 0,
 'AP_point_instr': 0.0,
 'AP_read_obj': 0,
 'AP_snowboard_instr': 0.0,
 'mAP_all': 6.294771562739988e-06,
 'mAP_thesis': 0.0}

: 

```
CUDA_VISIBLE_DEVICES=0 python entry.py evaluate \
            --conf_files configs/xdecoder/segvlp_focalt_lang.yaml \
            --overrides \
            COCO.INPUT.IMAGE_SIZE 1024 \
            MODEL.DECODER.CAPTIONING.ENABLED True \
            MODEL.DECODER.RETRIEVAL.ENABLED True \
            MODEL.DECODER.GROUNDING.ENABLED True \
            COCO.TEST.BATCH_SIZE_TOTAL 1 \
            COCO.TRAIN.BATCH_SIZE_TOTAL 1 \
            COCO.TRAIN.BATCH_SIZE_PER_GPU 1 \
            ADE20K.TEST.BATCH_SIZE_TOTAL 1 \
            VLP.TEST.BATCH_SIZE_TOTAL 32 \
            VLP.TRAIN.BATCH_SIZE_TOTAL 32 \
            VLP.TRAIN.BATCH_SIZE_PER_GPU 32 \
            MODEL.DECODER.HIDDEN_DIM 512 \
            MODEL.ENCODER.CONVS_DIM 512 \
            MODEL.ENCODER.MASK_DIM 512 \
            FP16 True \
            DONT_LOAD_MODEL False \
            PYLEARN_MODEL checkpoints/xdecoder_focalt_last.pt \
```